In [1]:
import numpy as np

In [24]:
# Contain the list of characters indices in the data
input_data = []
# Index of <space> set to 0
char_to_ind = {'<space>':0}
count = 1
with open('../data/train_chars.txt', 'r') as f:
    # Loop to index the char and store them inside the input
    for line in f:
        for c in line[:-1].split(' '):
            # Input data
            if c in char_to_ind:
                input_data.append(char_to_ind[c])
            else:
                char_to_ind[c] = count
                count += 1
                input_data.append(char_to_ind[c])


In [43]:
# Build the input matrix: (num_records, n-1)
# and output (num_records,1) which stores the output for the given (n-1)gram
input_matrix = np.zeros((len(input_data)-n, n-1))
output_matrix = np.zeros(len(input_data)-n)
for i in xrange(len(input_data)-n):
    # Countext is a (n-1)gram
    w = input_data[i:i+(n-1)]
    input_matrix[i,:] = w
    output_matrix[i] = (1 if input_data[i+n]==0 else 2)

In [90]:
def get_input(filename,n,char_to_ind=None):
    # Contain the list of characters indices in the data
    # initialized with a padding
    if n > 2:
        input_data = [1]*(n-2)
    else:
        input_data = []
    if char_to_ind is None:
        # Map each character to an index with
        # Index of <space> set to 0
        char_to_ind = {'<space>':0, '</s>':1}
        count = 2
    with open(filename, 'r') as f:
        # Loop to index the char and store them inside the input
        for line in f:
            for c in line[:-1].split(' '):
                # Input data
                if c in char_to_ind:
                    input_data.append(char_to_ind[c])
                else:
                    char_to_ind[c] = count
                    count += 1
                    input_data.append(char_to_ind[c])
    return input_data, char_to_ind

In [91]:
input_data, char_to_ind = get_input('../data/train_chars.txt',n=4)

In [68]:
input_data

[1,
 1,
 2,
 3,
 4,
 0,
 5,
 2,
 6,
 7,
 6,
 8,
 9,
 3,
 0,
 5,
 3,
 4,
 10,
 11,
 9,
 12,
 0,
 13,
 2,
 10,
 10,
 8,
 14,
 2,
 15,
 0,
 13,
 3,
 6,
 9,
 4,
 16,
 17,
 9,
 0,
 13,
 10,
 16,
 3,
 9,
 9,
 0,
 18,
 4,
 8,
 19,
 17,
 9,
 3,
 11,
 6,
 0,
 20,
 11,
 9,
 2,
 6,
 8,
 0,
 20,
 16,
 9,
 3,
 4,
 19,
 2,
 6,
 0,
 21,
 15,
 22,
 4,
 8,
 23,
 24,
 16,
 3,
 5,
 3,
 13,
 0,
 11,
 25,
 8,
 0,
 7,
 11,
 2,
 0,
 19,
 3,
 19,
 8,
 9,
 3,
 13,
 0,
 19,
 10,
 26,
 0,
 6,
 2,
 21,
 5,
 0,
 25,
 16,
 6,
 9,
 17,
 0,
 4,
 2,
 7,
 3,
 0,
 4,
 3,
 20,
 2,
 9,
 9,
 2,
 0,
 4,
 16,
 5,
 3,
 6,
 17,
 0,
 17,
 11,
 19,
 0,
 17,
 6,
 2,
 13,
 7,
 23,
 18,
 8,
 8,
 22,
 0,
 17,
 17,
 2,
 6,
 20,
 15,
 8,
 6,
 20,
 0,
 17,
 14,
 2,
 25,
 8,
 0,
 14,
 2,
 13,
 21,
 9,
 3,
 4,
 1,
 25,
 11,
 3,
 4,
 4,
 3,
 0,
 27,
 16,
 6,
 7,
 28,
 0,
 29,
 0,
 15,
 3,
 2,
 4,
 17,
 0,
 8,
 10,
 22,
 0,
 14,
 11,
 10,
 10,
 0,
 30,
 8,
 11,
 6,
 0,
 9,
 21,
 3,
 0,
 5,
 8,
 2,
 4,
 22,
 0,
 2,
 17,
 0,
 2,
 0,
 6,
 8,


In [77]:
def build_train_data(input_data, n=2):
    # Build the input matrix: (num_records, n-1)
    # and the output vector (num_records,1)
    # which stores the output for the given (n-1)gram
    input_matrix = np.zeros((len(input_data)-n, n-1))
    output_matrix = np.zeros(len(input_data)-n)
    for i in xrange(len(input_data)-n):
        # Countext is a (n-1)gram
        w = input_data[i:i+(n-1)]
        input_matrix[i,:] = w
        output_matrix[i] = (1 if input_data[i+(n-1)]==0 else 2)
    return input_matrix, output_matrix


In [94]:
input_data_test, char_to_ind_test = get_input('../data/valid_chars.txt',4, char_to_ind=char_to_ind)

In [95]:
len(char_to_ind_test)

49

In [89]:
for c in char_to_ind_test.keys():
    if c not in char_to_ind.keys():
        print c

In [88]:
len(char_to_ind)

49

In [78]:
input_matrix2, output_matrix2 = build_train_data(input_data, n=4)

In [84]:
input_matrix2[:15]

array([[ 1.,  1.,  2.],
       [ 1.,  2.,  3.],
       [ 2.,  3.,  4.],
       [ 3.,  4.,  0.],
       [ 4.,  0.,  5.],
       [ 0.,  5.,  2.],
       [ 5.,  2.,  6.],
       [ 2.,  6.,  7.],
       [ 6.,  7.,  6.],
       [ 7.,  6.,  8.],
       [ 6.,  8.,  9.],
       [ 8.,  9.,  3.],
       [ 9.,  3.,  0.],
       [ 3.,  0.,  5.],
       [ 0.,  5.,  3.]])

In [83]:
input_data[:19]

[1, 1, 2, 3, 4, 0, 5, 2, 6, 7, 6, 8, 9, 3, 0, 5, 3, 4, 10]

In [82]:
output_matrix2[:15]

array([ 2.,  2.,  1.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  1.,  2.,
        2.,  2.])

In [56]:
Counter(output_matrix2)

Counter({1.0: 100960, 2.0: 498940})

In [53]:
input_matrix2

array([[  1.,   2.,   3.,   0.],
       [  2.,   3.,   0.,   4.],
       [  3.,   0.,   4.,   1.],
       ..., 
       [ 18.,  10.,   9.,   9.],
       [ 10.,   9.,   9.,  10.],
       [  9.,   9.,  10.,   7.]])

In [31]:
from collections import Counter

# Store the count of the ngram
ngram_count = Counter()
n = 5
for i in xrange(len(input_data)-n):
    # Countext is a (n-1)gram
    w = input_data[i:i+n]
    # Change the last char of the ngram to a space indicator
    #if input_data[i+n] == 0:
    #    # Case space
    #    w.append(1)
    #else:
    #    w.append[0]
    ngram_count.update([tuple(w)])

In [34]:
# Storing the ngram_count: (n-1gram, count_no_space, count_space)
ngram_count_matrix = np.zeros((len(ngram_count), n+1))
for (i,(k,v)) in enumerate(ngram_count.iteritems()):
    ngram_count_matrix[i,:n] = k
    ngram_count_matrix[i,-1] = v

In [35]:
# Build the count matrix
ngram_count_matrix

array([[  9.,  21.,   0.,   1.,   4.,   3.],
       [ 15.,   5.,   2.,   0.,  31.,   1.],
       [  2.,   5.,   0.,   1.,  24.,   1.],
       ..., 
       [  2.,  21.,   0.,  24.,   2.,   5.],
       [  7.,   9.,   0.,  18.,   3.,   1.],
       [ 10.,  24.,   2.,  16.,   0.,   1.]])

In [27]:
char_to_ind

{'#': 46,
 '$': 39,
 '&': 38,
 "'": 33,
 '-': 22,
 '.': 32,
 '/': 48,
 '0': 37,
 '1': 34,
 '2': 41,
 '3': 40,
 '4': 42,
 '5': 36,
 '6': 44,
 '7': 45,
 '8': 43,
 '9': 35,
 '<': 27,
 '</s>': 26,
 '<space>': 0,
 '>': 28,
 'N': 29,
 '\\': 47,
 'a': 1,
 'b': 4,
 'c': 12,
 'd': 21,
 'e': 2,
 'f': 17,
 'g': 19,
 'h': 20,
 'i': 10,
 'j': 30,
 'k': 6,
 'l': 9,
 'm': 18,
 'n': 5,
 'o': 7,
 'p': 24,
 'q': 23,
 'r': 3,
 's': 16,
 't': 8,
 'u': 15,
 'v': 31,
 'w': 13,
 'x': 25,
 'y': 14,
 'z': 11}

In [28]:
len(input_data)

599905